<a href="https://colab.research.google.com/github/MahinArvind/DL-projects/blob/main/Alzheimers_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Alzheimer's Classification

##Importing Modules


In [35]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

## Loading Data

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mahinarvind","key":"aec012dfad33b5d2902ee02c95ad9d47"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images

 50% 17.0M/34.1M [00:01<00:01, 15.6MB/s]
100% 34.1M/34.1M [00:01<00:00, 29.4MB/s]


In [ ]:
from zipfile import ZipFile
filename = 'alzheimers-dataset-4-class-of-images.zip'

with ZipFile(filename, 'r') as zip:
  zip.extractall()
  print('Extracted')

Extracted


##Data Preprocessing

In [ ]:
_URL = '/content/Alzheimer_s Dataset'
PATH = os.path.join(os.path.dirname(path_to_zip), 'Alzheimer_s Dataset')

train_path = os.path.join(PATH, 'train')
valid_path = os.path.join(PATH, 'test')


In [37]:
BATCH_SIZE = 32
IMG_SIZE = [224, 224]

vgg = VGG16(input_shape=IMG_SIZE + [3], weights='imagenet', include_top=False)


58900480/58889256 [==============================] - 3s 0us/step


In [50]:
for layer in vgg.layers:
  layer.trainable = False
  

In [55]:
folders = glob('/content/Alzheimer_s Dataset/train/*')
print(len(folders))

4


In [64]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [66]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/Alzheimer_s Dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/Alzheimer_s Dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')
AUTOTUNE = tf.data.AUTOTUNE

train_set = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_set = test_dataset.prefetch(buffer_size=AUTOTUNE)

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [ ]:

# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')



/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
161/161 [==============================] - 3249s 20s/step - loss: 0.7112 - accuracy: 0.6926 - val_loss: 0.8018 - val_accuracy: 0.6474
Epoch 2/5
161/161 [==============================] - 3280s 20s/step - loss: 0.6526 - accuracy: 0.7135 - val_loss: 0.8926 - val_accuracy: 0.6255
Epoch 3/5
161/161 [==============================] - 3276s 20s/step - loss: 0.6218 - accuracy: 0.7280 - val_loss: 1.1812 - val_accuracy: 0.6333
Epoch 4/5
161/161 [==============================] - 3230s 20s/step - loss: 0.5994 - accuracy: 0.7383 - val_loss: 0.9491 - val_accuracy: 0.6396
Epoch 5/5
 98/161 [=================>............] - ETA: 17:01 - loss: 0.5359 - accuracy: 0.7685

Use buffered prefetching to load images from disk without having I/O become blocking.
[data performance guide](https://www.tensorflow.org/guide/data_performance)